# **Monta Google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/

Mounted at /content/drive
/content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02


# **Set Up CUDA**

Version Ubuntu, GPU version, GPU type, Plugin

In [ ]:
!lsb_release --all
!nvcc --version
!nvidia-smi 
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Sat Jan 22 14:21:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   


## Librerie per la K80


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


## Codice di test, risultato atteso = 8

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8



# **Caratteristiche Macchina**


## Caratteristiche scheda

In [ ]:
!nvidia-smi 

Sat Jan 22 12:21:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    77W / 149W | Function Not Found   |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Caratteristiche GPU

In [ ]:
%%cu

#include <stdio.h>
#include <stdlib.h>

void deviceQuery()
{
  cudaDeviceProp prop;
  int nDevices=0, i;
  cudaError_t ierr;

  ierr = cudaGetDeviceCount(&nDevices);
  if (ierr != cudaSuccess) { printf("Sync error: %s\n", cudaGetErrorString(ierr)); }



  for( i = 0; i < nDevices; ++i )
  {
     ierr = cudaGetDeviceProperties(&prop, i);
     printf("Device number: %d\n", i);
     printf("  Device name: %s\n", prop.name);
     printf("  Compute capability: %d.%d\n\n", prop.major, prop.minor);
     
     printf("  Clock Rate: %d kHz\n", prop.clockRate);
     printf("  Total SMs: %d \n", prop.multiProcessorCount);
     printf("  Shared Memory Per SM: %lu bytes\n", prop.sharedMemPerMultiprocessor);
     printf("  Registers Per SM: %d 32-bit\n", prop.regsPerMultiprocessor);
     printf("  Max threads per SM: %d\n", prop.maxThreadsPerMultiProcessor);
     printf("  L2 Cache Size: %d bytes\n", prop.l2CacheSize);
     printf("  Total Global Memory: %lu bytes\n", prop.totalGlobalMem);
     printf("  Memory Clock Rate: %d kHz\n\n", prop.memoryClockRate);
     
     
     printf("  Max threads per block: %d\n", prop.maxThreadsPerBlock);
     printf("  Max threads in X-dimension of block: %d\n", prop.maxThreadsDim[0]);
     printf("  Max threads in Y-dimension of block: %d\n", prop.maxThreadsDim[1]);
     printf("  Max threads in Z-dimension of block: %d\n\n", prop.maxThreadsDim[2]);

     printf("  Max blocks in X-dimension of grid: %d\n", prop.maxGridSize[0]);
     printf("  Max blocks in Y-dimension of grid: %d\n", prop.maxGridSize[1]);
     printf("  Max blocks in Z-dimension of grid: %d\n\n", prop.maxGridSize[2]);     
     
     printf("  Shared Memory Per Block: %lu bytes\n", prop.sharedMemPerBlock);
     printf("  Registers Per Block: %d 32-bit\n", prop.regsPerBlock);
     printf("  Warp size: %d\n\n", prop.warpSize);

  }
}

int main() {
    deviceQuery();
}

Device number: 0
  Device name: Tesla K80
  Compute capability: 3.7

  Clock Rate: 562000 kHz
  Total SMs: 13 
  Shared Memory Per SM: 114688 bytes
  Registers Per SM: 131072 32-bit
  Max threads per SM: 2048
  L2 Cache Size: 1572864 bytes
  Total Global Memory: 11996954624 bytes
  Memory Clock Rate: 2505000 kHz

  Max threads per block: 1024
  Max threads in X-dimension of block: 1024
  Max threads in Y-dimension of block: 1024
  Max threads in Z-dimension of block: 64

  Max blocks in X-dimension of grid: 2147483647
  Max blocks in Y-dimension of grid: 65535
  Max blocks in Z-dimension of grid: 65535

  Shared Memory Per Block: 49152 bytes
  Registers Per Block: 65536 32-bit
  Warp size: 32




In [ ]:
%cd /usr/local/cuda/samples
%cd 1_Utilities/bandwidthTest/
!make
!./bandwidthTest --mode-range --start=1000 --end=1000000 --increment=100000

/usr/local/cuda-9.2/samples
/usr/local/cuda-9.2/samples/1_Utilities/bandwidthTest
/usr/local/cuda-9.2/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_70,code=compute_70 -o bandwidthTest.o -c bandwidthTest.cu
/usr/local/cuda-9.2/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_70,code=compute_70 -o bandwidthTest bandwidthTest.o 
mkdir -p ../../bin/x86_64/linux/release
cp bandwidthTest ../../bin/

## Caratteristiche CPU

In [ ]:
!cat /proc/cpuinfo


processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

## Caratteristiche Memoria


In [ ]:
!cat /proc/meminfo

MemTotal:       13302920 kB
MemFree:          259780 kB
MemAvailable:   12346448 kB
Buffers:          281580 kB
Cached:         11540456 kB
SwapCached:            0 kB
Active:          3465628 kB
Inactive:        8872020 kB
Active(anon):     474156 kB
Inactive(anon):      468 kB
Active(file):    2991472 kB
Inactive(file):  8871552 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1976 kB
Writeback:             0 kB
AnonPages:        515676 kB
Mapped:           269880 kB
Shmem:              1200 kB
KReclaimable:     549592 kB
Slab:             600396 kB
SReclaimable:     549592 kB
SUnreclaim:        50804 kB
KernelStack:        5888 kB
PageTables:         7212 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    3797836 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       46004 kB
VmallocChunk:          0 kB
Percpu:          

## Disk Info




In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          79G   48G   31G  61% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  817M  59% /sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1        86G   52G   34G  61% /opt/bin/.nvidia
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            79G   50G   30G  63% /content/drive


# **Common Assignment 3**

## Global


In [ ]:
%cd /content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src/

/content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src


Ezecuzione con test senza creazione csv

In [ ]:
!nvcc -o global global.cu
!./global 10000 128
!./global 10000 256
!./global 10000 512
!./global 10000 1024

GridSize: 79
Kernel elapsed time 0.000185s 

TEST PASSED
GridSize: 40
Kernel elapsed time 0.000175s 

TEST PASSED
GridSize: 20
Kernel elapsed time 0.000149s 

TEST PASSED
GridSize: 10
Kernel elapsed time 0.000142s 

TEST PASSED


In [ ]:
!nvprof --print-gpu-trace ./global 100000000 1024 --benchmark 

GridSize: 97657
==23265== NVPROF is profiling process 23265, command: ./global 100000000 1024 --benchmark
Kernel elapsed time 0.072775s 

TEST PASSED
==23265== Profiling application: ./global 100000000 1024 --benchmark
==23265== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
433.54ms  58.115ms                    -               -         -         -         -  381.47MB  6.4103GB/s    Pageable      Device    Tesla K80 (0)         1         7  [CUDA memcpy HtoD]
491.94ms  4.4480us                    -               -         -         -         -      400B  85.762MB/s      Device           -    Tesla K80 (0)         1         7  [CUDA memset]
492.87ms  29.529ms          (97657 1 1)      (1024 1 1)         8        0B        0B         -           -           -           -    Tesla K80 (0)         1         7  countingSortKernel1(int*, int*, int

In [ ]:
!nvprof --metrics inst_integer ./global 100000000 1024

GridSize: 97657
==23326== NVPROF is profiling process 23326, command: ./global 100000000 1024
Kernel elapsed time 0.776683s 

TEST PASSED
==23326== Profiling application: ./global 100000000 1024
==23326== Profiling result:
==23326== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla K80 (0)"
    Kernel: countingSortKernel1(int*, int*, int)
          1                              inst_integer                      Integer Instructions   800001536   800001536   800001536
    Kernel: countingSortKernel2(int*, int)
          1                              inst_integer                      Integer Instructions        4239        4239        4239
    Kernel: countingSortKernel3(int*, int*, int*, int)
          1                              inst_integer                      Integer Instructions  1100001536  1100001536  1100001536


Ezecuzione con creazione csv

In [ ]:
!nvcc -o global_custom global_custom.cu
!./global_custom 10000000 1024

## Shared

In [ ]:
%cd /content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src/

/content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src


In [ ]:
!nvcc -o shared shared.cu
!./shared 100000 1024

GridSize: 98
Kernel elapsed time 0.000371s 
Mflops: 53925796.000000

TEST PASSED


In [ ]:
!nvcc -o shared shared.cu
!./shared 100000 1024
!./shared 100000 512
!./shared 100000 256
!./shared 100000 128

GridSize: 98
Kernel elapsed time 0.000214s 

TEST PASSED
GridSize: 196
Kernel elapsed time 0.000206s 

TEST PASSED
GridSize: 391
Kernel elapsed time 0.000201s 

TEST PASSED
GridSize: 782
Kernel elapsed time 0.000198s 

TEST PASSED


In [ ]:
!nvprof --print-gpu-trace ./shared 100000000 1024 --benchmark 

In [ ]:
!nvprof --metrics inst_integer ./shared 100000000 1024

GridSize: 97657
==23441== NVPROF is profiling process 23441, command: ./shared 100000000 1024
==23441== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
Kernel elapsed time 1.267833s 

TEST PASSED
==23441== Profiling application: ./shared 100000000 1024
==23441== Profiling result:
==23441== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla K80 (0)"
    Kernel: countingSortKernel2(int*, int)
          1                              inst_integer                      Integer Instructions        4239        4239        4239
    Kernel: countingSortKernel3(int*, int*, int*, int)
          1                              inst_integer                      Integer

Ezecuzione con creazione csv

In [ ]:
!nvcc -o shared_custom shared_custom.cu
!./shared_custom 10000000 1024

GridSize: 9766
Kernel elapsed time 0.006535s 


## Texture

In [ ]:
%cd /content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src/

In [ ]:
!nvcc -arch=sm_37 -o texture texture.cu
!./texture 100000 1024

GridSize: 98
Kernel elapsed time 0.000156s 

TEST PASSED


In [ ]:
!nvcc -o -arch=sm_37 -o texture texture.cu
!./texture 100000 1024
!./texture 100000 512
!./texture 100000 256
!./texture 100000 128

nvcc fatal   : redefinition of argument 'output-file'
GridSize: 98
Kernel elapsed time 0.000115s 

TEST PASSED
GridSize: 196
Kernel elapsed time 0.000128s 

TEST PASSED
GridSize: 391
Kernel elapsed time 0.000140s 

TEST PASSED
GridSize: 782
Kernel elapsed time 0.000099s 

TEST PASSED


In [ ]:
!nvprof --print-gpu-trace ./texture 100000000 1024 --benchmark 

In [ ]:
!nvprof --metrics inst_integer ./texture 100000000 1024

Ezecuzione con creazione del csv

In [ ]:
!nvcc -arch=sm_37 -o texture_custom texture_custom.cu
!./texture_custom 10000000 1024

# ***Analisi***

In [ ]:
%cd /content//drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src

/content/drive/MyDrive/STUDI/UNISA/CommonAssignment3-Team02/src


In [ ]:
!nvcc -o ./global_custom ./global_custom.cu
!./global_custom 102400000 1024
!nvcc -o ./shared_custom ./shared_custom.cu
!./shared_custom 102400000 1024
!nvcc -arch=sm_37 -o ./texture_custom ./texture_custom.cu
!./texture_custom 102400000 1024

!nvcc -o ./global_custom ./global_custom.cu
!./global_custom 102400000 512
!nvcc -o ./shared_custom ./shared_custom.cu
!./shared_custom 102400000 512
!nvcc -arch=sm_37 -o ./texture_custom ./texture_custom.cu
!./texture_custom 102400000 512

!nvcc -o ./global_custom ./global_custom.cu
!./global_custom 102400000 256
!nvcc -o ./shared_custom ./shared_custom.cu
!./shared_custom 102400000 256
!nvcc -arch=sm_37 -o ./texture_custom ./texture_custom.cu
!./texture_custom 102400000 256

!nvcc -o ./global_custom ./global_custom.cu
!./global_custom 102400000 128
!nvcc -o ./shared_custom ./shared_custom.cu
!./shared_custom 102400000 128
!nvcc -arch=sm_37 -o ./texture_custom ./texture_custom.cu
!./texture_custom 102400000 128

In [ ]:
!nvcc -o ./global_custom ./global_custom.cu
!nvcc -o ./shared_custom ./shared_custom.cu
!nvcc -arch=sm_37 -o ./texture_custom ./texture_custom.cu
for _ in range(100):
  !./global_custom 102400000 1024
  !./shared_custom 102400000 1024
  !./texture_custom 102400000 1024

for _ in range(100):
  !./global_custom 102400000 512
  !./shared_custom 102400000 512
  !./texture_custom 102400000 512

for _ in range(100):
  !./global_custom 102400000 256
  !./shared_custom 102400000 256
  !./texture_custom 102400000 256

for _ in range(100):
  !./global_custom 102400000 128
  !./shared_custom 102400000 128
  !./texture_custom 102400000 128

gcc: error: ./global_custom.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.
gcc: error: ./shared_custom.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.
gcc: error: ./texture_custom.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.
/bin/bash: ./global_custom: No such file or directory
/bin/bash: ./shared_custom: No such file or directory
/bin/bash: ./texture_custom: No such file or directory
/bin/bash: ./global_custom: No such file or directory
/bin/bash: ./shared_custom: No such file or directory
/bin/bash: ./texture_custom: No such file or directory
/bin/bash: ./global_custom: No such file or directory
/bin/bash: ./shared_custom: No such file or directory
/bin/bash: ./texture_custom: No such file o

In [ ]:
!nvcc -o global_custom global_custom.cu
!nvcc -o shared_custom shared_custom.cu
!nvcc -arch=sm_37 -o texture_custom texture_custom.cu
!echo "==============================================================GLOBAL====================================================================================="
!nvprof --metrics inst_integer ./global_custom 102400000 1024
!nvprof --metrics inst_integer ./global_custom 102400000 512
!nvprof --metrics inst_integer ./global_custom 102400000 256
!nvprof --metrics inst_integer ./global_custom 102400000 128
!echo "==============================================================SHARED====================================================================================="
!nvprof --metrics inst_integer ./shared_custom 102400000 1024
!nvprof --metrics inst_integer ./shared_custom 102400000 512
!nvprof --metrics inst_integer ./shared_custom 102400000 256
!nvprof --metrics inst_integer ./shared_custom 102400000 128
!echo "==============================================================TEXTURE====================================================================================="
!nvprof --metrics inst_integer ./texture_custom 102400000 1024
!nvprof --metrics inst_integer ./texture_custom 102400000 512
!nvprof --metrics inst_integer ./texture_custom 102400000 256
!nvprof --metrics inst_integer ./texture_custom 102400000 128

==============================================================GLOBAL=====================================================================================
GridSize: 100000
==21474== NVPROF is profiling process 21474, command: ./global_custom 102400000 1024
Kernel elapsed time 0.732695s 
==21474== Profiling application: ./global_custom 102400000 1024
==21474== Profiling result:
==21474== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla K80 (0)"
    Kernel: countingSortKernel1(int*, int*, int)
          1                              inst_integer                      Integer Instructions   819200000   819200000   819200000
    Kernel: countingSortKernel2(int*, int)
          1                              inst_integer                      Integer Instructions        4239        4239        4239
    Kernel: countingSortKernel3(int*, int*, int*, int)
          1                   